# 电子积分准备：傅里叶变换

重新初始化一下我们的体系。

In [1]:
from pyscf.pbc import gto

cell = gto.Cell()
cell.atom='''
C 0.000000000000   0.000000000000   0.000000000000
C 1.685068664391   1.685068664391   1.685068664391
'''
cell.a = '''
0.000000000, 3.370137329, 3.370137329
3.370137329, 0.000000000, 3.370137329
3.370137329, 3.370137329, 0.000000000'''
cell.unit = 'B'
cell.basis = 'gth-szv'
cell.pseudo = 'gth-pade'
cell.build()

In [2]:
S_ref = cell.pbc_intor('int1e_ovlp')

## 重叠积分：傅里叶变换

$$ \phi(\boldsymbol{G}) = \int e^{-i \boldsymbol{G}\cdot\boldsymbol{r}} \phi(\boldsymbol{r}) \mathrm{d} \boldsymbol{r} $$

In [3]:
mesh = [10,10,10]
grids = cell.get_uniform_grids(mesh)

In [4]:
Gv = cell.get_Gv(mesh=mesh)
print(Gv.shape)

(1000, 3)


## 电子-核吸引能积分



In [16]:
from pyscf.pbc.tools import fft, ifft, cutoff_to_mesh
import numpy as np

def get_nuc(cell, mesh, grids):
    R = cell.atom_coords()
    Gv = cell.get_Gv(mesh=mesh)
    SI = np.exp(-1j*np.einsum('zs,xs->zx', R, Gv))
    rho_nuc_G = -np.einsum('z,zx->x', cell.atom_charges(), SI)

    aoR = cell.pbc_eval_gto('GTOval', grids)
    nao = aoR.shape[1]
    pair_ao_real_space = np.einsum('xi,xj->xij', aoR.conj(), aoR)
    pair_ao_G_space = ifft(pair_ao_real_space.reshape(-1,nao**2).T,
                           mesh).T.reshape(-1,nao,nao) * cell.vol

    w = 1./cell.vol
    G2 = np.einsum('xs,xs->x', Gv, Gv)
    coulG = 4 * np.pi / G2
    coulG[G2 == 0] = 0

    v = w * np.einsum('x,x,xij->ij', rho_nuc_G, coulG, pair_ao_G_space)
    return v

enuc = get_nuc(cell, mesh, grids)

ke_cutoff = 20.0
_mesh = cutoff_to_mesh(cell.lattice_vectors(), ke_cutoff)
print(_mesh)
_grids = cell.get_uniform_grids(_mesh)
enuc_cut = get_nuc(cell, _mesh, _grids)
print(np.linalg.norm(enuc_cut - enuc))

[8 8 8]
0.1435936658459252


/tmp/ipykernel_2243445/3290720319.py:18: RuntimeWarning: divide by zero encountered in divide
  coulG = 4 * np.pi / G2
